In [1]:
import ee
from component.scripts.GEE_wrappers import GEE_extent
from component.scripts.derive_SM import create_out_name
ee.Initialize()

<IPython.core.display.Javascript object>

In [2]:
def export_sm(image, file_name):
    description = f"fileexp_{file_name}"

    task = ee.batch.Export.image.toDrive(
        image=image.__getattribute__("ESTIMATED_SM"),
        description=description,
        fileNamePrefix=file_name,
        scale=150,
        region=image.roi.getInfo()["coordinates"],
        maxPixels=1e13,
    )
    task.start()
    return task, file_name

In [3]:
test_coords_chips = [
[101.81744576590408, 0.2869119197606414, 102.30001067617684, 0.669037480331793], [102.30001067617683, 0.16903270683388336, 102.80001067617683, 0.669037480331793], [102.80001067617683, 0.30066643754335914, 103.1085784557263, 0.6690364871822352], [101.80001067617683, 0.6690311121559859, 102.30001067617684, 1.169040066354326], [102.30001067617684, 0.6690311121559859, 102.80001067617684, 0.935023369243529], [102.80001067617683, 0.6690311121559859, 102.95128286303692, 0.7397827247042105], [102.16045110331572, 1.1690378684394913, 102.20857581471485, 1.2306851406088202]]
len(test_coords_chips)

7

In [5]:
chip_to_test = 3

In [7]:
# get GEE interface
GEE_interface = GEE_extent(*test_coords_chips[chip_to_test])

# retrieve S1
GEE_interface.get_S1(
    2022,
    1,
    9,
    tempfilter=True,
    applylcmask=False,
    mask_globcover=True,
    masksnow=False,
    ascending=False,
)

# retrieve GLDAS
GEE_interface.get_gldas()

if GEE_interface.GLDAS_IMG is not None:
    outname = create_out_name(
        GEE_interface.S1_DATE.year,
        GEE_interface.S1_DATE.month,
        GEE_interface.S1_DATE.day,
        "test_image" + f"chip_{chip_to_test}",
    )

    # get Globcover
    GEE_interface.get_globcover()

    # get the SRTM
    GEE_interface.get_terrain()

    # Estimate soil moisture
    GEE_interface.estimate_SM()

In [8]:
scale = 100
task = ee.batch.Export.image.toDrive(
    image=GEE_interface.__getattribute__("ESTIMATED_SM"),
    description=f"{outname}_{scale}",
    fileNamePrefix=f"{outname}_{scale}",
    scale=scale,
    region=GEE_interface.roi.getInfo()["coordinates"],
    maxPixels=1e13,
)
task.start()